<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/cricket_match_data_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab Cell 1: Install dependencies and unzip dataset
!pip install pandas requests
!unzip "/content/archive (1).zip" -d "/content/ipl_data"

import pandas as pd
import requests
import time
from google.colab import files
import os

# Step 1: Load and process IPL match data
matches_file = "/content/ipl_data/matches.csv"
deliveries_file = "/content/ipl_data/deliveries.csv"

# Load matches.csv
matches_df = pd.read_csv(matches_file)
print("Matches columns:", matches_df.columns.tolist())

# Filter for seasons 2020–2024
matches_df = matches_df[matches_df['season'].isin(['2020/21', '2021', '2022', '2023', '2024'])]

# Standardize columns
matches_df = matches_df.rename(columns={
    'id': 'Match ID',
    'date': 'Date',
    'venue': 'Venue',
    'city': 'City',
    'team1': 'Team1',
    'team2': 'Team2',
    'toss_winner': 'Toss Winner',
    'toss_decision': 'Toss Decision',
    'winner': 'Match Result',
})

# Ensure Date is in YYYY-MM-DD format
matches_df['Date'] = pd.to_datetime(matches_df['Date']).dt.strftime('%Y-%m-%d')

# Load deliveries.csv for innings details
deliveries_df = pd.read_csv(deliveries_file)
print("Deliveries columns:", deliveries_df.columns.tolist())

# Calculate innings scores and wickets
innings_summary = deliveries_df.groupby(['match_id', 'inning']).agg({
    'batsman_runs': 'sum',
    'player_dismissed': lambda x: x.notna().sum()
}).reset_index()

# Pivot to get first and second innings
innings_pivot = innings_summary.pivot(index='match_id', columns='inning', values=['batsman_runs', 'player_dismissed'])
innings_pivot.columns = [f"{col[0]}_{col[1]}" for col in innings_pivot.columns]
innings_pivot = innings_pivot.reset_index()

# Rename columns
innings_pivot = innings_pivot.rename(columns={
    'batsman_runs_1': 'First Innings Score',
    'player_dismissed_1': 'First Innings Wickets',
    'batsman_runs_2': 'Second Innings Score',
    'player_dismissed_2': 'Second Innings Wickets',
})

# Determine batting teams
batting_teams = deliveries_df.groupby(['match_id', 'inning'])['batting_team'].first().reset_index()
batting_teams_pivot = batting_teams.pivot(index='match_id', columns='inning', values='batting_team').reset_index()
batting_teams_pivot = batting_teams_pivot.rename(columns={
    1: 'First Innings Team',
    2: 'Second Innings Team',
})

# Merge innings details with matches
matches_df = matches_df.merge(innings_pivot, left_on='Match ID', right_on='match_id', how='left')
matches_df = matches_df.merge(batting_teams_pivot, left_on='Match ID', right_on='match_id', how='left')

# Format score/wickets
matches_df['First Innings Score'] = matches_df['First Innings Score'].astype(str) + '/' + matches_df['First Innings Wickets'].astype(str)
matches_df['Second Innings Score'] = matches_df['Second Innings Score'].astype(str) + '/' + matches_df['Second Innings Wickets'].astype(str)

# Standardize Match Result
matches_df['Match Result'] = matches_df.apply(
    lambda row: f"{row['Match Result']} won" if pd.notna(row['Match Result']) else 'No result', axis=1
)

# Infer City for missing values and standardize venue-to-city mapping
venue_to_city = {
    'Sheikh Zayed Stadium': 'Abu Dhabi',
    'Dubai International Cricket Stadium': 'Dubai',
    'Sharjah Cricket Stadium': 'Sharjah',
    'Zayed Cricket Stadium, Abu Dhabi': 'Abu Dhabi',
    'MA Chidambaram Stadium, Chepauk, Chennai': 'Chennai',
    'Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh': 'Mohali',
    'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow': 'Lucknow',
    'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 'Hyderabad',
    'Himachal Pradesh Cricket Association Stadium, Dharamsala': 'Dharamsala',
    'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam': 'Visakhapatnam',
    'M Chinnaswamy Stadium, Bengaluru': 'Bengaluru',
    'Wankhede Stadium, Mumbai': 'Mumbai',
    'Narendra Modi Stadium, Ahmedabad': 'Ahmedabad',
    'Arun Jaitley Stadium, Delhi': 'Delhi',
    'Eden Gardens, Kolkata': 'Kolkata',
    'Sawai Mansingh Stadium, Jaipur': 'Jaipur',
    'Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur': 'Mullanpur',
    'Barsapara Cricket Stadium, Guwahati': 'Guwahati',
    'Maharashtra Cricket Association Stadium, Pune': 'Pune',
    'Brabourne Stadium, Mumbai': 'Mumbai',
    'Dr DY Patil Sports Academy, Mumbai': 'Mumbai',
}

matches_df['City'] = matches_df.apply(
    lambda row: venue_to_city.get(row['Venue'], row['City']), axis=1
)

# Select relevant columns
ipl_df = matches_df[[
    'Match ID', 'Date', 'Venue', 'City', 'Team1', 'Team2', 'Toss Winner', 'Toss Decision',
    'First Innings Team', 'First Innings Score', 'First Innings Wickets',
    'Second Innings Team', 'Second Innings Score', 'Second Innings Wickets', 'Match Result'
]]

print("✅ IPL DataFrame shape:", ipl_df.shape)
print(ipl_df.head())

# Step 2: Define city-to-country mapping for Visual Crossing
city_country = {
    'Abu Dhabi': 'United Arab Emirates',
    'Dubai': 'United Arab Emirates',
    'Sharjah': 'United Arab Emirates',
    'Chennai': 'India',
    'Mohali': 'India',
    'Lucknow': 'India',
    'Hyderabad': 'India',
    'Dharamsala': 'India',
    'Visakhapatnam': 'India',
    'Bengaluru': 'India',
    'Mumbai': 'India',
    'Ahmedabad': 'India',
    'Delhi': 'India',
    'Kolkata': 'India',
    'Jaipur': 'India',
    'Mullanpur': 'India',
    'Guwahati': 'India',
    'Pune': 'India',
}

# Step 3: Fetch weather data using Visual Crossing
API_KEY = 'ADUVHU7PFYMT68R3EQPL4QQ8N'  # Your provided API key
weather_recs = []
for _, row in ipl_df.iterrows():
    city = row['City']
    if pd.isna(city) or city not in city_country:
        print(f"⚠️ city missing or unmapped for {row['Venue']} ({city}); skipping")
        continue
    location = f"{city},{city_country[city]}"
    date = row['Date']

    url = (
        f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{date}/{date}"
        f"?key={API_KEY}&include=days&elements=tempmax,tempmin,precip,windspeed"
    )
    for attempt in range(3):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json().get('days', [{}])[0]
            if not data:
                print(f"⚠️ no weather for {city} ({row['Venue']}) on {date}")
                break

            weather_recs.append({
                'Date': date,
                'Venue': row['Venue'],
                'Max_Temp_C': data.get('tempmax'),
                'Min_Temp_C': data.get('tempmin'),
                'Total_Precip_mm': data.get('precip', 0.0),
                'Max_Wind_kmh': float(data.get('windspeed', 0.0)) * 3.6,  # Convert mph to km/h
            })
            print(f"🌦️ Weather for {city} ({row['Venue']}) on {date}")
            break
        except (requests.RequestException, KeyError, ValueError) as e:
            print(f"⚠️ Error fetching weather for {city} ({row['Venue']}) on {date}: {e}")
            time.sleep(1)
    time.sleep(0.2)

weather_df = pd.DataFrame(weather_recs)
print("\n✅ Weather DataFrame shape:", weather_df.shape)
print(weather_df.head())

# Step 4: Merge and save
final_df = pd.merge(
    ipl_df,
    weather_df,
    on=['Date', 'Venue'],
    how='left'
)
final_df.to_csv('IPL_Match_Weather_Data_2020_2024.csv', index=False)
print(f"\n✅ Done! 'IPL_Match_Weather_Data_2020_2024.csv' with {final_df.shape[0]} rows created.")
print(final_df.head())

# Download the file
files.download('IPL_Match_Weather_Data_2020_2024.csv')